In [4]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', lambda x: '{:,.0f}'.format(x))

# 1. Read Data
assets_df = pd.read_excel('Assets.xlsx')
debts_df = pd.read_excel('Debts.xlsx')

# Print the loaded data
print("Assets Data:")
print(assets_df)
print("\nDebts Data:")
print(debts_df)

# Calculate the sum of assets and debts
sum_assets = assets_df['total_price_per_unit'].sum()
print(f"\nSum of assets: {int(sum_assets):,}")
sum_debts = debts_df['total_price'].sum()
print(f"Sum of debts: {int(sum_debts):,}")

# 2. Assignment Algorithm
# Sort assets and debts by priority
assets_df = assets_df.sort_values(by=['priority'], ascending=[True]).reset_index(drop=True)
debts_df = debts_df.sort_values(by=['priority'], ascending=[True]).reset_index(drop=True)

assignments = []

def assign_assets_to_debts(assets_df, debts_df):
    for debt_index, debt in debts_df.iterrows():
        remaining_debt = debt['total_price']
        total_assigned_value_to_debt = 0  # Initialize a variable to track the total assigned value to each debt
        for asset_index, asset in assets_df.iterrows():
            if asset['available_units'] == 0:
                continue
            asset_total_value = asset['meter_per_unit'] * asset['per_meter_price'] * asset['available_units']
            if asset_total_value <= remaining_debt:
                units_to_assign = asset['available_units']
                remaining_debt -= asset_total_value
                assets_df.at[asset_index, 'available_units'] = 0
                total_assigned_value_to_debt += asset_total_value  # Accumulate the assigned value
                assignments.append((units_to_assign, asset['code'], debt['code'], asset_total_value, debt['Name']))
            else:
                units_to_assign = int(remaining_debt / (asset['meter_per_unit'] * asset['per_meter_price']))
                if units_to_assign > asset['available_units']:
                    units_to_assign = asset['available_units']
                assigned_value = units_to_assign * asset['meter_per_unit'] * asset['per_meter_price']
                remaining_debt -= assigned_value
                assets_df.at[asset_index, 'available_units'] -= units_to_assign
                total_assigned_value_to_debt += assigned_value  # Accumulate the assigned value
                if units_to_assign > 0:
                    assignments.append((units_to_assign, asset['code'], debt['code'], assigned_value, debt['Name']))
            if remaining_debt <= 0:
                break
        debts_df.at[debt_index, 'remaining_debt'] = remaining_debt
        debts_df.at[debt_index, 'assigned_value'] = total_assigned_value_to_debt  # Add total assigned value to debt

assign_assets_to_debts(assets_df, debts_df)

# 3. Print Assignments
assignments_df = pd.DataFrame(assignments, columns=['Units', 'Asset Code', 'Debt Code', 'Assigned Value', 'Debt Name'])
print("\nAssignments:")
print(assignments_df)

# Calculate the total value of remaining assets and debts
total_value_assets = (assets_df['total_price_per_unit'] * assets_df['available_units']).sum()
total_value_debts = debts_df['remaining_debt'].sum()

# Print remaining units of assets and debts
remaining_assets_df = assets_df[['code', 'available_units']]
remaining_debts_df = debts_df[['code', 'Name', 'remaining_debt', 'assigned_value']]
remaining_debts_df.loc[:, 'assigned_value'] = remaining_debts_df['assigned_value'].apply(lambda x: '{:,.0f}'.format(x))

# Calculate Units Used per Asset and Their Total Value
units_used_per_asset = assignments_df.groupby('Asset Code').agg(
    Total_Units_Used=('Units', 'sum'),
    Total_Value_Assigned=('Assigned Value', 'sum')
).reset_index()

# Format the values in the new DataFrame
units_used_per_asset['Total_Value_Assigned'] = units_used_per_asset['Total_Value_Assigned'].apply(lambda x: f"{x:,}")

# Merge with the remaining assets DataFrame
merged_assets_df = pd.merge(
    remaining_assets_df,
    units_used_per_asset,
    left_on='code',
    right_on='Asset Code',
    how='left'
).drop(columns=['Asset Code'])

print("\nRemaining Assets and Units Used:")
print(merged_assets_df)

print("\nRemaining Amount and Assigned Value of Debts:")
print(remaining_debts_df)

# Calculate Total Value of Assignments
total_assigned_value = assignments_df['Assigned Value'].sum()
total_assigned_value_df = pd.DataFrame({'Total Assigned Value': [f"{total_assigned_value:,}"]})
print("\nTotal Value of Assignments:")
print(total_assigned_value_df)


Assets Data:
          code  available_units  meter_per_unit  per_meter_price  \
0  MahmoudAbad               13             157        150000000   
1    Shohada75                2              75        120000000   
2   Shohada100                2             100        120000000   
3   Shohada120                4             120        120000000   

   total_price_per_unit  priority  
0          306150000000         1  
1           18000000000         2  
2           24000000000         4  
3           57600000000         3  

Debts Data:
                code                  Name  total_price  priority
0      D5-1403-2-1-8  Sazeh Gostar Iranian  28700000000         5
1     D12-1403-2-2-7       Foolad Mehregan  21300000000        12
2    D1-1403-2-13-10     Sazeh Sazan Behin  27100000000         1
3   D11-1403-2-11-11        Beton Parsabad  41500000000        11
4     D4-1403-2-12-7    Peymankarane Pouya  12000000000         4
5    D13-1403-1-15-4            Omran Paya  44800000000  